In [13]:
import pandas as pd
import os
import datetime
import subprocess
import re
from omegaconf import OmegaConf
import pathlib

In [14]:
USER = 'ruyogagp'

TEMPLATE_YAML = 'riskiano/experiments/jakob/config_charite/MET_MTS_DeepSurv_HS_excl_CANCER.yaml' # template yaml to use
TRAIN_SCRIPT = 'medical_interpretability/experiments/train.py'   # python train script to use
EXPERIMENT_NAME = 'correlation_case_p-0.30'

In [15]:
os.makedirs(f'/home/{USER}/tmp/{EXPERIMENT_NAME}/job_submissions', exist_ok=True)
os.makedirs(f'/home/{USER}/tmp/{EXPERIMENT_NAME}/job_configs', exist_ok=True)

In [16]:
def submit(path, job_name, job_script, time_stamp=None, gpu=False):
    if not time_stamp:
        time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    script_path_long = f'./{job_name}_{time_stamp}.sh'

    with open(script_path_long, 'w') as outfile: outfile.write(job_script)
    script_path = f'./{job_name}.sh'
    try:
        os.unlink(script_path)
    except FileNotFoundError: # because we cannot overwrite symlinks directly
        pass
    os.symlink(os.path.realpath(script_path_long), script_path)

    print(job_script)
    print('\n\nSubmission:\n===========\n')

    mode = 'graphical' if gpu else 'normal'
    sub_cmd = f'bsub -q {mode} -m "eils-dgx1-02" -gpu "num=1:mode=shared" < {script_path}'
    print(sub_cmd)

    ret = subprocess.run(sub_cmd, shell=True, cwd=os.getcwd(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(ret.stdout.decode())
    print(ret.stderr.decode())

    jobid = re.findall(r'<[0-9]+>', ret.stdout.decode())[0][1:-1]
    return jobid

In [17]:
def make_job_script(user, experiment_id, memory=20, walltime=10, cores=20):

    job_script_str = f'''
    #! /bin/sh
    #BSUB -e /home/{user}/errors/job_%J.err
    #BSUB -o /home/{user}/errors/job_%J.out
    #BSUB -q "graphical"
    #BSUB -R "rusage[mem={memory * 1000}]"
    #BSUB -M {memory * 1000}
    #BSUB -W {walltime}:00
    #BSUB -n {cores}
    #BSUB -J {experiment_id}

    source /home/ruyogagp/miniconda/etc/profile.d/conda.sh
    conda activate /data/analysis/ag-reils/ruyogagp/envs/new
    cd /home/ruyogagp/medical_interpretability/

    python -m experiments.train experiment_id={experiment_id}

    '''
    return job_script_str

In [18]:
config = dict(config_dir='/home/ruyogagp/medical_interpretability/source/config',
              config_name = 'interpretability',
              job_name='correlation_case')

## Run as group

In [19]:
exp = 'linear_revised'
experiment_ids = [f'{exp}_1', f'{exp}_2', f'{exp}_3']
for experiment_id in experiment_ids:
    job_script = make_job_script(user='ruyogagp', experiment_id=experiment_id, memory=20, walltime=10, cores=12)
    jobid = submit(path=f"job_submissions",
                   gpu=True,
                   job_name=config["job_name"],
                   job_script=job_script)


    #! /bin/sh
    #BSUB -e /home/ruyogagp/errors/job_%J.err
    #BSUB -o /home/ruyogagp/errors/job_%J.out
    #BSUB -q "graphical"
    #BSUB -R "rusage[mem=20000]"
    #BSUB -M 20000
    #BSUB -W 10:00
    #BSUB -n 12
    #BSUB -J linear_revised_1

    source /home/ruyogagp/miniconda/etc/profile.d/conda.sh
    conda activate /data/analysis/ag-reils/ruyogagp/envs/new
    cd /home/ruyogagp/medical_interpretability/

    python -m experiments.train experiment_id=linear_revised_1

    


Submission:

bsub -q graphical -m "eils-dgx1-02" -gpu "num=1:mode=shared" < ./correlation_case.sh
Job <531131> is submitted to queue <graphical>.



    #! /bin/sh
    #BSUB -e /home/ruyogagp/errors/job_%J.err
    #BSUB -o /home/ruyogagp/errors/job_%J.out
    #BSUB -q "graphical"
    #BSUB -R "rusage[mem=20000]"
    #BSUB -M 20000
    #BSUB -W 10:00
    #BSUB -n 12
    #BSUB -J linear_revised_2

    source /home/ruyogagp/miniconda/etc/profile.d/conda.sh
    conda activate /data/analysis/ag-reils/ruyogagp

## Run single experiment

In [12]:
experiment_id = 'simpsons_revised'
job_script = make_job_script(user='ruyogagp', experiment_id=experiment_id, memory=20, walltime=10, cores=12)
jobid = submit(path=f"job_submissions",
               gpu=True,
               job_name=config["job_name"],
               job_script=job_script)


    #! /bin/sh
    #BSUB -e /home/ruyogagp/errors/job_%J.err
    #BSUB -o /home/ruyogagp/errors/job_%J.out
    #BSUB -q "graphical"
    #BSUB -R "rusage[mem=20000]"
    #BSUB -M 20000
    #BSUB -W 10:00
    #BSUB -n 12
    #BSUB -J simpsons_revised

    source /home/ruyogagp/miniconda/etc/profile.d/conda.sh
    conda activate /data/analysis/ag-reils/ruyogagp/envs/new
    cd /home/ruyogagp/medical_interpretability/

    python -m experiments.train experiment_id=simpsons_revised

    


Submission:

bsub -q graphical -m "eils-dgx1-02" -gpu "num=1:mode=shared" < ./correlation_case.sh
Job <530126> is submitted to queue <graphical>.


